
## Machine Learning item x item
### def **recomendacion_juego( *`id de producto`* )**:
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado
falta similaridad indices .pkl

In [ ]:

df_ML = pd.read_parquet("./0 Dataset/F_df_funciones.parquet")

def recomendacion_juego(item_id: int):
    with open("./0 Dataset/similaridad_indices.pkl", "rb") as archivo:
        pickle.load(archivo)
    
    try:
        with open('./0 Dataset/similaridad_indices.pkl', 'rb') as f:
            similaridad_indices = pickle.load(f)
    except FileNotFoundError:
        matrix_sparse = csr_matrix(pd.crosstab(df_ML['item_id'], df_ML['user_id'], values=df_ML['reviews_recommend'], aggfunc='sum').fillna(0))
        simil = cosine_similarity(matrix_sparse)
        similaridad_indices = {item_id: simil[item_index].argsort()[-6:-1][::-1] for item_index, item_id in enumerate(df_ML['item_id'].unique().tolist())}
        with open('./0 Dataset/similaridad_indices.pkl', 'wb') as f:
            pickle.dump(similaridad_indices, f)
        del matrix_sparse, simil
    
    try:
        item_similar = [df_ML['item_id'].unique().tolist()[i] for i in similaridad_indices[item_id]]
        item_similar_nombre = [(df_ML[df_ML['item_id'] == item]['item_id'].values[0], df_ML[df_ML['item_id'] == item]['item_name'].values[0]) for item in item_similar]
        return {"Recomendaciones para Item ID": item_id, "Juegos Recomendados": item_similar_nombre}
    except KeyError:
        return {"error": f"Item ID {item_id} not found."}
    finally:
        # Liberar memoria utilizando el recolector de basura
        gc.collect()

recomendacion_juego(500)